#### Check if the data requires some preprocessing. At the very least, you should have the data scaled for k-means and PCA. Take into account that you can always unscale the data for better interpretation of the results.


In [1]:
import pandas as pd

df = pd.read_csv("customer_data_large.csv")
print(df.head())  
print(df.info())
print(df.isnull().sum())
print(df.describe()) 

   Education  Kidhome  Recency  MntWines  MntFruits  MntSweetProducts  \
0          3        0       58       635         88                88   
1          3        1       38        11          1                 1   
2          3        0       26       426         49                21   
3          3        1       26        11          4                 3   
4          5        1       94       173         43                27   

   MntGoldProds  NumWebPurchases  NumStorePurchases  NumWebVisitsMonth  Age  \
0            88                8                  4                  7   65   
1             6                1                  2                  5   63   
2            42                8                 10                  4   57   
3             5                7                  4                  6   38   
4            15                5                  6                  5   41   

   Years_customer  Num_AcceptedCmp  MntFishMeatProdcts  
0              10            

In [2]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

print(df_scaled.head())

   Education   Kidhome   Recency  MntWines  MntFruits  MntSweetProducts  \
0  -0.457341 -0.825281  0.305096  0.982341   1.554427          1.473303   
1  -0.457341  1.031809 -0.385769 -0.870089  -0.636292         -0.631912   
2  -0.457341 -0.825281 -0.800288  0.361896   0.572380         -0.147954   
3  -0.457341  1.031809 -0.800288 -0.870089  -0.560750         -0.583516   
4   1.536541  1.031809  1.548652 -0.389170   0.421296         -0.002767   

   MntGoldProds  NumWebPurchases  NumStorePurchases  NumWebVisitsMonth  \
0      0.850944         1.028070          -0.551928          -0.026234   
1     -0.729510        -1.254743          -1.166559          -0.517841   
2     -0.035652         1.028070           1.291966          -0.763645   
3     -0.748784         0.701954          -0.551928          -0.272038   
4     -0.556046         0.049722           0.062703          -0.517841   

        Age  Years_customer  Num_AcceptedCmp  MntFishMeatProdcts  
0  1.292790        1.503387         0